<a href="https://colab.research.google.com/github/shaikadish/imdbProject/blob/main/model_training_and_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Mount to google drive
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/GitHub/IMDB_project/imdbProject

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/GitHub/IMDB_project/imdbProject


In [5]:
!pip install transformers

from transformers import BertTokenizer
import pandas as pd
import torch

In [4]:
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('fabriceyhc/bert-base-uncased-imdb', do_lower_case=True)

Loading BERT tokenizer...


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/321 [00:00<?, ?B/s]

In [5]:
review_data=pd.read_csv('review_data_learning.csv')
reviews=review_data['0'].to_list()
ratings=review_data['1'].to_list()

In [ ]:
review=review_data.iloc[0,0]

print(tokenizer.encode(review, add_special_tokens=True,max_length=512,truncation=True))
# NEED TO PROCESS DATA TO BE <512 WORDS IN LENGTH.
review='[CLS] '+review.replace('.',' [SEP]')+'[PAD]'
# Print the original sentence.
print(' Original: ', review)

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(review))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(review)))

[101, 2004, 1037, 2143, 7089, 1045, 2064, 1005, 1056, 2393, 24256, 9107, 1996, 18804, 2143, 2466, 2023, 2143, 2003, 1012, 2023, 2003, 1996, 2472, 1005, 1055, 14822, 2000, 2010, 2219, 2397, 2388, 1010, 2029, 4265, 2013, 21901, 1005, 1055, 4295, 1012, 2002, 1005, 1055, 2478, 2010, 2219, 7436, 4222, 2143, 2013, 2889, 1010, 1000, 2336, 1997, 3267, 1000, 2004, 1037, 2067, 7265, 6199, 2005, 1996, 2466, 1010, 2029, 2191, 2149, 2514, 1996, 2693, 2003, 1037, 4516, 1010, 2030, 2012, 2560, 1037, 2995, 2466, 1012, 9588, 2175, 3995, 2003, 3225, 2125, 2007, 2019, 6429, 18804, 3325, 1010, 2007, 1037, 2143, 9338, 10886, 2010, 2143, 2012, 1037, 6765, 1010, 4760, 1996, 3193, 2013, 2307, 12113, 1012, 1996, 12127, 2044, 11682, 4455, 2010, 3080, 2388, 1010, 3225, 2000, 9015, 2013, 21901, 1005, 1055, 11703, 19500, 3974, 2491, 1010, 2021, 2025, 2079, 2172, 2008, 2016, 3475, 1005, 1056, 2583, 2000, 2131, 2041, 1997, 4390, 2145, 1012, 1998, 2023, 3475, 1005, 1056, 1996, 2143, 11153, 2069, 3291, 1012, 1996, 247

In [6]:
import torch

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for review in reviews:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        review,                      # Sentence to encode.
                        truncation=True,
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)
labels = torch.tensor(ratings)

# Print sentence 0, now as a list of IDs.
print('Original: ', reviews[0])
print('Token IDs:', input_ids[0])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  As a film lover I can't help immensely enjoying the meta film story this film is. This is the director's homage to his own late mother, which suffered from Alzheimer's disease. He's using his own Oscar nominated film from 1991, "Children Of Nature" as a backdraft for the story, which make us feel the move is a documentary, or at least a true story.Mama Gogo is starting off with an amazing meta experience, with a film maker presenting his film at a premiere, showing the view from great angles. The filmmaker after wards calls his older mother, starting to suffer from Alzheimer's decease losing control, but not do much that she isn't able to get out of trouble still. And this isn't the film makers only problem. The Islandic economy is on it's way to a crisis, and the film maker is experiencing that Icelandic people aren't too much into his movies. Mama is going into oblivion and does dangerous things, while the film maker, his wife and his sisters are going nuts being accused o

In [9]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.7 * len(dataset))+1
val_size = int(0.5*(len(dataset) - train_size))
test_size = int(0.5*(len(dataset) - train_size))

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size,test_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
print('{:>5,} test samples'.format(test_size))

117,344 training samples
25,145 validation samples
25,145 test samples


In [8]:
from torch.utils.data import DataLoader

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            shuffle=True,
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            batch_size = batch_size # Evaluate with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
test_dataloader = DataLoader(
            test_dataset, # The validation samples.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [11]:
import pickle


outfile = open('val_dataloader','wb')
pickle.dump(validation_dataloader,outfile)
outfile.close()

outfile = open('train_dataloader','wb')
pickle.dump(train_dataloader,outfile)
outfile.close()

outfile = open('test_dataloader','wb')
pickle.dump(test_dataloader,outfile)
outfile.close()

In [6]:
import pickle

infile = open('val_dataloader','rb')
validation_dataloader = pickle.load(infile)
infile.close()

infile = open('test_dataloader','rb')
test_dataloader = pickle.load(infile)
infile.close()

infile = open('train_dataloader','rb')
train_dataloader = pickle.load(infile)
infile.close()

In [7]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels =  1, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [18]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [21]:
model.train()
for step, batch in enumerate(train_dataloader):
  if step>4:
    break

  model.zero_grad()

  b_input_ids = batch[0].cuda()
  b_input_mask = batch[1].cuda()
  b_labels = batch[2].cuda()
  
  loss, logits = model(b_input_ids, 
                      token_type_ids=None, 
                      attention_mask=b_input_mask, 
                      labels=b_labels,
                      return_dict=False)
  print(loss)

RuntimeError: ignored

In [17]:
b_labels

tensor([ 8,  3,  9,  1,  5,  2,  9,  5,  5,  5,  7,  9,  3,  2, 10,  5],
       device='cuda:0')

In [ ]:
import torch
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved

f

123058176

In [14]:
!nvidia-smi

Wed May 11 19:05:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    26W /  70W |  11898MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [15]:
del model
torch.cuda.empty_cache()

RuntimeError: ignored